# Create metrics_config.toml

In this notebook we create the metrics_config.toml file. This file contains the SQL queries we want to do on the Delft-FIAT output to extract the data needed for the infographics.

In [1]:
from pathlib import Path
import pandas as pd
from os import makedirs
import tomli_w

In [2]:
model_root = Path('c:/Repositories/DT-flood/FloodAdapt_database/Humber/')

scenario_name = 'empty_event_current_no_measures'

fiat_out_path = model_root / 'output' / 'scenarios' / scenario_name / 'Impacts' / ('Impacts_detailed_'+scenario_name+'.csv')

metrics_path = model_root / 'static' / 'templates' / 'infometrics' / 'metrics_config.toml'

In [3]:
fiat_out = pd.read_csv(fiat_out_path)

In [4]:
fiat_out

,WKT,Object ID,index_right,Primary Object Type,Secondary Object Type,Max Potential Damage: Structure,Max Potential Damage: Content,Ground Floor Height,Extraction Method,Damage Function: Structure,Damage Function: Content,Ground Elevation,Inundation Depth,Reduction Factor,Damage: Structure,Damage: Content,Total Damage
0,"POLYGON ((-0.6243402 53.5692152,-0.6243111 53....",26636960,24,industrial,industrial,1.078175e+06,1.617263e+06,0,centroid,industrial,industrial,22.777802,NaN,NaN,NaN,NaN,0.00
1,"POLYGON ((-0.6243402 53.5692152,-0.6243111 53....",26636960,24,industrial,industrial,1.078175e+06,1.617263e+06,0,centroid,industrial,industrial,22.777802,NaN,NaN,NaN,NaN,0.00
2,"POLYGON ((-0.6492636 53.5919747,-0.6493065 53....",26638125,21,residential,residential,3.348470e+05,1.674235e+05,0,centroid,residential,residential,37.496788,NaN,NaN,NaN,NaN,0.00
3,"POLYGON ((-0.6515431 53.5794368,-0.6515448 53....",35392645,23,residential,residential,7.065027e+05,3.532513e+05,0,centroid,residential,residential,43.017334,NaN,NaN,NaN,NaN,0.00
4,"POLYGON ((-0.6608791 53.5916054,-0.6610171 53....",35392648,21,residential,residential,3.585271e+06,1.792636e+06,0,centroid,residential,residential,48.825794,NaN,NaN,NaN,NaN,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136325,"POLYGON ((-0.0369071 53.5572645,-0.0367261 53....",11726731,16553,residential,residential,7.861787e+05,3.930893e+05,0,centroid,residential,residential,14.252323,NaN,NaN,NaN,NaN,0.00
136326,"POLYGON ((-0.1916957 53.5852767,-0.1914899 53....",14618789,15854,residential,residential,7.086432e+05,3.543216e+05,0,centroid,residential,residential,8.863322,NaN,NaN,NaN,NaN,0.00
136327,"POLYGON ((-0.1141542 53.5584711,-0.1141673 53....",14643365,14451,residential,residential,6.715145e+05,3.357573e+05,0,centroid,residential,residential,3.059216,0.27,1.0,90654.46,45327.23,135981.69
136328,"POLYGON ((-0.0934216 53.5585673,-0.0933837 53....",14720336,14451,residential,residential,5.135883e+05,2.567941e+05,0,centroid,residential,residential,6.726851,NaN,NaN,NaN,NaN,0.00


In [5]:
print(fiat_out['Primary Object Type'].unique())
print(fiat_out['Secondary Object Type'].unique())

['industrial' 'residential' 'commercial']
['industrial' 'residential' 'retail' 'commercial']


## Querie

A querie needs to consist of the following:
- `name`: ID of the querie
- `description`: explanation of what to calculate
- `select`: SQL command as str how to calculate quantity (e.g. COUNT, SUM)
- `filter`: SQL command as str which data to select (e.g. which column)
- `long_name`
- `show_in_metrics_table`

In [6]:
querie_list = []

Automate metric generation more. Make widgets with dropdown menu storing values into parameters (one for building type, one for stat to be queried, threshold etc) and make function that takes these parameters in and generates the right SQL querie and block in toml file

In [9]:
querie_single = {
    "name": "TotalDamageEvent",
    "description": "Total damage to buildings",
    "select": "SUM('Total Damage')",
    "filter": "",
    "long_name": "Total building damage",
    "show_in_metrics_table": 'True'
}


In [10]:
querie_list.append(querie_single)
print(querie_list)

[{'name': 'ResidentialCount', 'description': 'Number of Residential properties affected', 'select': 'COUNT(*)', 'filter': "'Primary Object Type' in ('residential') AND 'Inundation Depth' > 0", 'long_name': 'Total building damage', 'show_in_metrics_table': 'True'}, {'name': 'TotalDamageEvent', 'description': 'Total damage to buildings', 'select': "SUM('Total Damage')", 'filter': '', 'long_name': 'Total building damage', 'show_in_metrics_table': 'True'}]


## Aggregation

A list of some aggregation to be used. Any entry should have a corresponding entry in the site.toml file under fiat.aggregation

In [11]:
agg_list = []

## Write toml file

In [12]:
metric_dict = {
    "queries": querie_list,
}

if len(agg_list) > 0:
    metric_dict.update({"aggregateBy": agg_list})
    
print(metric_dict)

{'queries': [{'name': 'ResidentialCount', 'description': 'Number of Residential properties affected', 'select': 'COUNT(*)', 'filter': "'Primary Object Type' in ('residential') AND 'Inundation Depth' > 0", 'long_name': 'Total building damage', 'show_in_metrics_table': 'True'}, {'name': 'TotalDamageEvent', 'description': 'Total damage to buildings', 'select': "SUM('Total Damage')", 'filter': '', 'long_name': 'Total building damage', 'show_in_metrics_table': 'True'}]}


In [13]:
if not metrics_path.parent.exists():
    makedirs(metrics_path.parent)

with open(metrics_path,'wb') as f:
    tomli_w.dump(metric_dict,f)